In [1]:
import os
%pwd

'c:\\Users\\dengz\\Desktop\\Projects\\cnn_classifier\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\dengz\\Desktop\\Projects\\cnn_classifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PredictionConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_num_classes: int

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath) 
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_predict_config(self) -> PredictionConfig:
            predict_config = PredictionConfig(
            path_of_model=Path("artifacts/training/model.pt"),
            training_data=Path("artifacts/data_ingestion/Chest-CT-Scan-data"),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_num_classes=self.params.CLASSES
        )
            return predict_config



In [6]:
import torch
import torch.nn as nn
from torchvision import transforms, models, datasets
from PIL import Image
from pathlib import Path
from cnnClassifier.entity.config_entity import TrainingConfig


In [7]:
class Prediction:
    def __init__(self,
                 filename: str,
                 config: TrainingConfig,
                 device: torch.device = None):
        """
        :param filename: Path to the image to predict
        :param config: TrainingConfig instance for model construction and loading
        :param device: torch.device, optional. Automatically selects GPU/CPU if not specified
        """
        self.filename = filename
        self.config = config
        self.device = device or torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Load class names (subfolder names from ImageFolder)
        self.class_names = datasets.ImageFolder(self.config.training_data).classes
        print(f"Class index mapping: {{i: name for i, name in enumerate(self.class_names)}}")

        # Build and load model
        self.model = self._load_model()

    def _load_model(self) -> nn.Module:
        # Model architecture should match training configuration
        model = models.resnet18(weights=None)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, self.config.params_num_classes)

        # Load trained weights
        state_dict = torch.load(self.config.path_of_model, map_location=self.device)
        model.load_state_dict(state_dict)
        model.to(self.device).eval()
        return model

    def predict(self) -> list:
        # Preprocessing pipeline (should match training transformations)
        preprocess = transforms.Compose([
            transforms.Resize(self.config.params_image_size[:2]),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
        ])

        # Load and process image
        img = Image.open(self.filename).convert('RGB')
        input_tensor = preprocess(img).unsqueeze(0).to(self.device)

        # Inference
        with torch.no_grad():
            outputs = self.model(input_tensor)
            probs = torch.softmax(outputs, dim=1).squeeze(0)
            idx = int(probs.argmax())
            confidence = float(probs[idx])

        label = self.class_names[idx]
        print(f"Predicted result: index {idx} -> class '{label}', confidence {confidence:.3f}")

        # Return structured result
        return [{
            "index": idx,
            "label": label,
            "confidence": confidence
        }]

In [12]:
try:
    file = Path(r"C:\Users\dengz\Desktop\Projects\cnn_classifier\artifacts\data_ingestion\Chest-CT-Scan-data\adenocarcinoma\000000 (6).png")
    config = ConfigurationManager()
    eval_config = config.get_predict_config()
    evaluation = Prediction(file,eval_config)
    evaluation.predict()
except Exception as e:
    raise e

[2025-04-27 21:21:35,917: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-27 21:21:35,920: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-27 21:21:35,921: INFO: common: created directory at: artifacts]
Class index mapping: {i: name for i, name in enumerate(self.class_names)}


C:\Users\dengz\AppData\Local\Temp\ipykernel_18504\3969338253.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(self.config.path_of_model, map_loca

Predicted result: index 0 -> class 'adenocarcinoma', confidence 0.998
